In [15]:
# Load data ke dalam data frame 
import pandas as pd

# Spesifikasi encoding diperlukan karena data tidak menggunakan UTF-8
df = pd.read_csv('dataset gabungan-270.csv', encoding='latin-1')

df.head()

,Review,Sentimen
0,Kecewa banget. Saya pesan yang free pouch tapi...,Negatif
1,"Kali ini pengemasan tidak safe,, sepertinya tu...",Negatif
2,paket datangnya barang sudah pecah dan sy retu...,Negatif
3,"ga ada sabun nya, ga sesuai deskripsi",Negatif
4,ga sesuai harapan,Negatif


PREPROCESSING

In [16]:
# PROSES CASE FOLDING (menjadikan semua huruf di kolom Review menjadi huruf kecil)
df['Review'] = df['Review'].str.lower()

print('Case Folding Result: \n')
df.head()

Case Folding Result: 



,Review,Sentimen
0,kecewa banget. saya pesan yang free pouch tapi...,Negatif
1,"kali ini pengemasan tidak safe,, sepertinya tu...",Negatif
2,paket datangnya barang sudah pecah dan sy retu...,Negatif
3,"ga ada sabun nya, ga sesuai deskripsi",Negatif
4,ga sesuai harapan,Negatif


In [17]:
# import hasil case folding menjadi csv
df.to_csv("casefolding_270.csv")

In [18]:
# CLEANING
import re # regex library

def remove_tweet_special(text):
    # remove tab, new line, and back slice
    text = text.replace('\\t'," ").replace('\\n', " ").replace('\\u', " ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")

df['Review'] = df['Review'].apply(remove_tweet_special)

# remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

df['Review'] = df['Review'].apply(remove_number)

# remove punctuation and replace with space
def remove_punctuation(text):
    return re.sub(r'[.,]', ' ', text)

df['Review'] = df['Review'].apply(remove_punctuation)

# remove punctuation
def remove_symbol(text):
    # Menghapus simbol-simbol tidak standar dan menggantinya dengan spasi
    cleaned_text = re.sub(r'[^\w\s]', ' ', text)
    
    # Menghapus multiple whitespace
    cleaned_text = re.sub('\s+', ' ', cleaned_text).strip()
    
    return cleaned_text

df['Review'] = df['Review'].apply(remove_symbol)

# Menentukan ambang batas panjang string acak
threshold_length = 20

# Fungsi untuk menghapus string acak berdasarkan panjangnya
def hapus_string_acak_dengan_panjang(text):
    # Memeriksa panjang string dan menghapus jika melebihi ambang batas
    return ' '.join(word for word in text.split() if len(word) <= threshold_length)

# Menggunakan fungsi untuk menghapus string acak pada kolom 'Text'
df['Review'] = df['Review'].apply(hapus_string_acak_dengan_panjang)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

df['Review'] = df['Review'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

df['Review'] = df['Review'].apply(remove_whitespace_multiple)

# remove single char
def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

df['Review'] = df['Review'].apply(remove_single_char)

def remove_laughter(text):
    laughter_patterns = r'\b((ha)+h*|(he)+h*|(hi)+h*|(wk)+w*k*|(eh)+e*|(ah)+a*|(ih)+i*|(kw)+k*w*|(hem)+m*)\b'
    return re.sub(laughter_patterns, '', text, flags=re.IGNORECASE)

df['Review'] = df['Review'].apply(remove_laughter)

print('Cleaning Result : \n') 
print(df.head())

Cleaning Result : 

                                              Review Sentimen
0  kecewa banget saya pesan yang free pouch tapi ...  Negatif
1  kali ini pengemasan tidak safe sepertinya tutu...  Negatif
2  paket datangnya barang sudah pecah dan sy retu...  Negatif
3               ga ada sabun nya ga sesuai deskripsi  Negatif
4                                  ga sesuai harapan  Negatif


In [19]:
# import hasil cleaning menjadi csv
df.to_csv("cleaning_270.csv")

In [20]:
# REMOVE DUPLICATE
# Menghapus baris yang memiliki duplikat data pada kolom review
df = df.drop_duplicates(subset=['Review'])
df = df.reset_index(drop=True)
# Menghapus baris yang kosong
df = df.dropna(subset=['Review'])
# Menghapus baris yang hanya berisi spasi atau whitespace
df = df[df['Review'].str.strip() != '']
df.head()

,Review,Sentimen
0,kecewa banget saya pesan yang free pouch tapi ...,Negatif
1,kali ini pengemasan tidak safe sepertinya tutu...,Negatif
2,paket datangnya barang sudah pecah dan sy retu...,Negatif
3,ga ada sabun nya ga sesuai deskripsi,Negatif
4,ga sesuai harapan,Negatif


In [21]:
# import hasil remove duplicate menjadi csv
df.to_csv("removedup_270.csv")

In [22]:
# NORMALIZATION
import pandas as pd
import re

slang_dictionary = pd.read_csv('colloquial-indonesian-lexicon2.csv')
slang_dict = pd.Series(slang_dictionary['formal'].values,index=slang_dictionary['slang']).to_dict()

slang_dictionary.head()

,slang,formal,In-dictionary,context,category1,category2,category3
0,woww,wow,1.0,wow,elongasi,0,0
1,aminn,amin,1.0,Selamat ulang tahun kakak tulus semoga panjang...,elongasi,0,0
2,met,selamat,1.0,Met hari netaas kak!? Wish you all the best @t...,abreviasi,0,0
3,netaas,menetas,1.0,Met hari netaas kak!? Wish you all the best @t...,afiksasi,elongasi,0
4,keberpa,keberapa,0.0,Birthday yg keberpa kak?,abreviasi,0,0


In [23]:
# Normalisasi kata menggunakan kamus colloquial-indonesian-lexicon2.csv
def Slangwords(text, slang_dict):
    for word in text.split():
        if word in slang_dict.keys():
            # menambahkan \b untuk menandakan batas kata di sekitar kata slang
            text = re.sub(r'\b{}\b'.format(re.escape(word)), slang_dict[word], text)
    
    text = re.sub('@[\w]+', '', text)
    return text

df['Review_norm'] = df['Review'].apply(lambda x: Slangwords(x, slang_dict))
print(df.head())

                                              Review Sentimen  \
0  kecewa banget saya pesan yang free pouch tapi ...  Negatif   
1  kali ini pengemasan tidak safe sepertinya tutu...  Negatif   
2  paket datangnya barang sudah pecah dan sy retu...  Negatif   
3               ga ada sabun nya ga sesuai deskripsi  Negatif   
4                                  ga sesuai harapan  Negatif   

                                         Review_norm  
0  kecewa banget saya pesan yang free pouch tapi ...  
1  kali ini pengemasan tidak safe sepertinya tutu...  
2  paket datangnya barang sudah pecah dan saya re...  
3       enggak ada sabun nya enggak sesuai deskripsi  
4                              enggak sesuai harapan  


In [24]:
# import hasil replace slang words menjadi csv
df.to_csv("normalization_270.csv")

In [25]:
# PROSES TOKENIZING (word_tokenize() untuk memecah string kedalam tokens)

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

# Tokenizing
# NLTK word tokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df['review_tokens'] = df['Review_norm'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(df.head())

Tokenizing Result : 

                                              Review Sentimen  \
0  kecewa banget saya pesan yang free pouch tapi ...  Negatif   
1  kali ini pengemasan tidak safe sepertinya tutu...  Negatif   
2  paket datangnya barang sudah pecah dan sy retu...  Negatif   
3               ga ada sabun nya ga sesuai deskripsi  Negatif   
4                                  ga sesuai harapan  Negatif   

                                         Review_norm  \
0  kecewa banget saya pesan yang free pouch tapi ...   
1  kali ini pengemasan tidak safe sepertinya tutu...   
2  paket datangnya barang sudah pecah dan saya re...   
3       enggak ada sabun nya enggak sesuai deskripsi   
4                              enggak sesuai harapan   

                                       review_tokens  
0  [kecewa, banget, saya, pesan, yang, free, pouc...  
1  [kali, ini, pengemasan, tidak, safe, sepertiny...  
2  [paket, datangnya, barang, sudah, pecah, dan, ...  
3  [enggak, ada, sabun, nya, e

In [26]:
# import hasil tokenizing menjadi csv
df.to_csv("tokenizing_270.csv")

In [27]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
# PROSES FILTERING (Stopword Removal) menggunakan library sastrawi

class CustomStopWordRemover:
    def __init__(self, stop_words):
        self.stop_words = stop_words

    def remove(self, text):
        words = text.split()
        filtered_words = [word for word in words if word.lower() not in self.stop_words]
        return ' '.join(filtered_words)

# Tentukan lokasi file stopword Bahasa Indonesia
stopword_file = 'stopwords_indonesian.txt'

# Baca isi file stopword
with open(stopword_file, 'r', encoding='utf-8') as file:
    stopwords = file.read().splitlines()

# Kata-kata yang ingin dikecualikan
exceptions = ["belum", "tidak", "tanpa"]

# Buat daftar stopword baru tanpa kata-kata pengecualian
custom_stopwords = [word.lower() for word in stopwords if word.lower() not in exceptions]

# Buat instance custom stopword remover
custom_stopword_remover = CustomStopWordRemover(custom_stopwords)

# remove stopword pada list token
def stopwords_removal(words):
    filtered_words = [custom_stopword_remover.remove(w) for w in words]
    filtered_words = [word for word in filtered_words if word != '']  # Memfilter kata-kata kosong
    return filtered_words

df['review_tokens_SR'] = df['review_tokens'].apply(stopwords_removal) 

print(df.head())

                                              Review Sentimen  \
0  kecewa banget saya pesan yang free pouch tapi ...  Negatif   
1  kali ini pengemasan tidak safe sepertinya tutu...  Negatif   
2  paket datangnya barang sudah pecah dan sy retu...  Negatif   
3               ga ada sabun nya ga sesuai deskripsi  Negatif   
4                                  ga sesuai harapan  Negatif   

                                         Review_norm  \
0  kecewa banget saya pesan yang free pouch tapi ...   
1  kali ini pengemasan tidak safe sepertinya tutu...   
2  paket datangnya barang sudah pecah dan saya re...   
3       enggak ada sabun nya enggak sesuai deskripsi   
4                              enggak sesuai harapan   

                                       review_tokens  \
0  [kecewa, banget, saya, pesan, yang, free, pouc...   
1  [kali, ini, pengemasan, tidak, safe, sepertiny...   
2  [paket, datangnya, barang, sudah, pecah, dan, ...   
3  [enggak, ada, sabun, nya, enggak, sesuai, des

In [28]:
# import hasil stopword removal menjadi csv
df.to_csv("stopword_stopword kecuali-no stemming.csv")